# dbt Cloud dbtc Quick Overview - Calling Jobs

### In this notebook we do the following:
- Import the dbtc package (dbt Cloud API SDK)
- Kick off a dbt Cloud Job
- Retrieve the status of that job
- Log the status of the job after it's finished


dbtc docs can be found [here](https://dbtc.dpguthrie.com/0.3/guide/cloud/)


#### Installing dbtc

In [3]:
from dbtc import dbtCloudClient
import time

#### Defing the service token

In [11]:
# dbt_cloud_service_token = 'dbts_DQ_5d1kOyBEGisNJPq0oPRm_JQW2uJT1oSOPWCqQcizEZkxAg0Jm6DIQ=='

#### Enabling the client

In [12]:
dbt_cloud_client = dbtCloudClient(service_token=dbt_cloud_service_token)

#### Setting up the inputs 

In [13]:
dbt_cloud_account_id = 123456
dbt_cloud_job_id = 569845
job_payload = {
  "cause": "Kicked off from Jupyter Notebook by Steve"
}

#### Kicking off the job with polling set to True

In [14]:
# triggering dbt cloud job 
job_trigger = dbt_cloud_client.cloud.trigger_job(account_id = dbt_cloud_account_id, 
                                      job_id = dbt_cloud_job_id, 
                                      payload = job_payload, 
                                      should_poll = True,
                                      poll_interval = 10)


[16:26:48] Status: "Queued", Elapsed time: 0s, View here:                                             ]8;id=157857;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=569151;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1367\1367]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100845044/                              

[16:26:59] Status: "Running", Elapsed time: 10.0s, View here:                                         ]8;id=351767;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=152971;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1375\1375]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100845044/                              

[16:27:09] Status: "Running", Elapsed time: 21.0s, View here:                                         ]8;id=111967;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=629305;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1375\1375]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100845044/                              

[16:27:20] Status: "Success", Elapsed time: 31.0s, View here:                                         ]8;id=709961;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=43310;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1375\1375]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100845044/                              

#### Kicking off the job with polling set to False

In [15]:
job_trigger = dbt_cloud_client.cloud.trigger_job(account_id = dbt_cloud_account_id, 
                                      job_id = dbt_cloud_job_id, 
                                      payload = job_payload, 
                                      should_poll = False)

[16:27:27] Status: "Queued", Elapsed time: 0s, View here:                                             ]8;id=709165;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=615173;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1367\1367]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100845086/                              

In [16]:
# defining the run status map
run_status_map = {
  1:  'Queued',
  2:  'Starting',
  3:  'Running',
  10: 'Success',
  20: 'Error',
  30: 'Cancelled',
}

# getting the job run id from the run
job_run_id = job_trigger['data']['id']

# defining run status
run_status = dbt_cloud_client.cloud.get_run(account_id = dbt_cloud_account_id, 
                               run_id = job_run_id)['data']['status']

# running a while loop to get the job status 
while run_status in [1, 2, 3]:
    
    # sleep for 5 seconds
    time.sleep(5)
    
    # retrieving the updated status
    run_status = dbt_cloud_client.cloud.get_run(account_id = dbt_cloud_account_id, 
                               run_id = job_run_id)['data']['status']

    # printing the status
    print(f"dbt Cloud job run {job_run_id} curretly has a status of {run_status_map[run_status]}")
    
    
# print the final status of the job 
print(f"dbt Cloud job run {job_run_id} finished with the status {run_status_map[run_status]}")

dbt Cloud job run 100845086 curretly has a status of Running
dbt Cloud job run 100845086 curretly has a status of Running
dbt Cloud job run 100845086 curretly has a status of Running
dbt Cloud job run 100845086 curretly has a status of Success
dbt Cloud job run 100845086 finished with the status Success


#### Kicking off the job with step over-rides

Example Payload - API docs are [here](https://documenter.getpostman.com/view/14183654/UVsSNiXC#148fd718-6440-46f2-998a-8a05eb5df4ad)

```{
  "cause": "Kicked off from Airflow",
  "git_branch": "staging",
  "schema_override": "dbt_cloud_pr_123",
  "dbt_version_override": "1.0.1",
  "target_name_override": "staging",
  "timeout_seconds_override": 3000,
  "generate_docs_override": true,
  "threads_override": 8,
  "steps_override": [
    "dbt seed",
    "dbt run --fail-fast",
    "dbt test --fail-fast"
  ]
}'```

In [17]:
# setting inputs
dbt_cloud_account_id = 123456
dbt_cloud_job_id = 569845
job_payload = {
  "cause": "Kicked off from Jupyter Notebook by Steve",
  "steps_override": [
    "dbt seed"
  ]
}

# triggering dbt cloud job 
job_trigger = dbt_cloud_client.cloud.trigger_job(account_id = dbt_cloud_account_id, 
                                      job_id = dbt_cloud_job_id, 
                                      payload = job_payload, 
                                      should_poll = True,
                                      poll_interval = 10)

[16:30:52] Status: "Queued", Elapsed time: 0s, View here:                                             ]8;id=708750;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=505401;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1367\1367]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100846129/                              

[16:31:02] Status: "Running", Elapsed time: 10.0s, View here:                                         ]8;id=980614;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=961882;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1375\1375]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100846129/                              

[16:31:13] Status: "Success", Elapsed time: 21.0s, View here:                                         ]8;id=809332;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py\admin.py]8;;\:]8;id=414498;file:///opt/homebrew/lib/python3.9/site-packages/dbtc/client/admin.py#1375\1375]8;;\
           https://cloud.getdbt.com/next/deploy/96504/projects/161955/runs/100846129/                              

#### Retrieving info from a run 
- can be used to retieve errors

In [20]:
# pulling the job run id from the object that kicked off the job above
job_run_id = job_trigger['data']['id']

# defining run status
run_info = dbt_cloud_client.cloud.get_run(account_id = dbt_cloud_account_id, 
                               run_id = job_run_id)

# printing the run info
print(run_info)

# example run info below

Example run info returned as dict:

</br>

```{'status': {'code': 200, 'is_success': True, 'user_message': 'Success!', 'developer_message': ''}, 'data': {'id': 100846129, 'trigger_id': 101561260, 'account_id': 123456, 'environment_id': 56789, 'project_id': 569888, 'job_definition_id': 569845, 'status': 10, 'dbt_version': '1.2.0-latest', 'git_branch': 'main', 'git_sha': '6231f4a8bc92452454fea8d20bb0d84dd4b32407', 'status_message': None, 'owner_thread_id': None, 'executed_by_thread_id': 'dbt-run-569888', 'deferring_run_id': None, 'artifacts_saved': True, 'artifact_s3_path': 'prod/runs/569888/artifacts/target', 'has_docs_generated': False, 'has_sources_generated': False, 'notifications_sent': True, 'blocked_by': [], 'scribe_enabled': True, 'created_at': '2022-11-28 21:30:52.210228+00:00', 'updated_at': '2022-11-28 21:31:11.610886+00:00', 'dequeued_at': '2022-11-28 21:30:52.248203+00:00', 'started_at': '2022-11-28 21:30:57.237571+00:00', 'finished_at': '2022-11-28 21:31:11.208356+00:00', 'last_checked_at': '2022-11-28 21:31:11.514719+00:00', 'last_heartbeat_at': '2022-11-28 21:30:57.223247+00:00', 'should_start_at': '2022-11-28 21:30:52.210228+00:00', 'trigger': None, 'job': None, 'environment': None, 'run_steps': [], 'status_humanized': 'Success', 'in_progress': False, 'is_complete': True, 'is_success': True, 'is_error': False, 'is_cancelled': False, 'href': 'https://cloud.getdbt.com/next/deploy/123456/projects/161955/runs/569845/', 'duration': '00:00:18', 'queued_duration': '00:00:05', 'run_duration': '00:00:13', 'duration_humanized': '18 seconds', 'queued_duration_humanized': '5 seconds', 'run_duration_humanized': '13 seconds', 'created_at_humanized': '5 minutes, 17 seconds ago', 'finished_at_humanized': '4 minutes, 58 seconds ago', 'job_id': 569845, 'is_running': None}}```